Notebook 4: Dune Migration
Animating changes in topography over time and computing erosion rates


In [ ]:
# Purpose: Use r.contour to extract contour.
# l is the shoreline proxy elevation
# c is the threshold to exclude small anomalies
l = 0.8
c = 400
dates = [19961016, 19971002, 19980907, 19990909,
19990918, 2001, 20040925, 20051126, 20080327]
for date in dates:
rast = ’NH_’+str(date)+’_1m’
grass.run_commands( ’r.contour’, input=rast,
output=

In [ ]:
# Purpose: Smooth extracted shoreline.
v.generalize input=NH_2008_08m output=NH_2008_08m_sm \
method=sliding_averaging threshold=0 look_ahead=51

In [ ]:
# visualization